In [1]:
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import time

In [2]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)

start_time = time.time()
data=pd.read_csv('covtype.csv')
print("Read time = {} seconds".format(time.time() - start_time))

In [ ]:
print('Data Dimension:')
print('Number of Records:', data.shape[0])
print('Number of Features:', data.shape[1])

In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

In [5]:
data.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,0.448865,0.051434,0.436074,0.063627,0.005217,0.012952,0.008301,0.021335,0.002749,0.011316,0.000181,0.000308,0.001974,0.056168,0.021359,0.051584,0.030001,0.001031,0.000005,0.004897,0.005890,0.003268,0.006921,0.015936,0.001442,0.057439,0.099399,0.036622,0.000816,0.004456,0.001869,0.001628,0.198356,0.051927,0.044175,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060,2.051471
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,0.497379,0.220882,0.495897,0.244087,0.072039,0.113066,0.090731,0.144499,0.052356,0.105775,0.013442,0.017550,0.044387,0.230245,0.144579,0.221186,0.170590,0.032092,0.002272,0.069804,0.076518,0.057077,0.082902,0.125228,0.037950,0.232681,0.299197,0.187833,0.028551,0.066605,0.043193,0.040318,0.398762,0.221879,0.205483,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791,1.396504
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [6]:
from sklearn.model_selection import train_test_split

target = data.Cover_Type
train = data.drop(['Cover_Type'], axis = 1)

start_time = time.time()
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.25, random_state=16)
print("Train test split time = {} seconds".format(time.time() - start_time))

In [72]:
from sklearn.ensemble import ExtraTreesClassifier

trees = ExtraTreesClassifier(n_estimators = 100, max_features=10)
start_time = time.time()
trees.fit(x_train, y_train)
print("ExtraTreesClassifier train time = {} seconds".format(time.time() - start_time))

ExtraTreesClassifier(max_features=10, n_estimators=200)

In [63]:
trees_result = trees.predict(x_test)

In [64]:
print(trees_result)

[2 2 6 ... 1 2 2]


In [65]:
sub_test = pd.DataFrame(data={'Id': x_test.index.values, 'Cover_Type': trees_result})
sub_test.to_csv("Results.csv", index=False)

In [66]:
print(sub_test)

            Id  Cover_Type
0       433102           2
1        87269           2
2       245488           6
3       189165           1
4       307227           1
...        ...         ...
174299   77818           2
174300  233801           2
174301  528012           1
174302  502555           2
174303  549162           2

[174304 rows x 2 columns]


In [73]:
start_time = time.time()
trees.score(x_test, y_test)
print("ExtraTreesClassifier score time = {} seconds".format(time.time() - start_time))

0.9584999965577303

In [81]:
from sklearn.neighbors import KNeighborsClassifier

for neighbors in range(1, 25):
    knn = KNeighborsClassifier(n_neighbors=neighbors)
    start_time = time.time()
    knn.fit(x_train, y_train)
    print("KNeighborsClassifier{} train time = {} seconds".format(neighbors, time.time() - start_time))
    start_time = time.time()
    print(neighbors, ": ", knn.score(x_test, y_test))
    print("KNeighborsClassifier{} score time = {} seconds".format(neighbors, time.time() - start_time))

KeyboardInterrupt: 

In [84]:
import xgboost

model = xgboost.XGBClassifier()
start_time = time.time()
model.fit(x_train, y_train)
print("XGBClassifier train time = {} seconds".format(time.time() - start_time))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [86]:
from sklearn.metrics import accuracy_score
start_time = time.time()
print("XGBoost accuracy: ", accuracy_score(y_test, [round(item) for item in model.predict(x_test)]))
print("XGBoost accuracy time = {} seconds".format(time.time() - start_time))

XGBoost accuracy:  0.8691868670526598


In [90]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 100, max_features=10)
start_time = time.time()
model.fit(x_train, y_train)
print("RandomForestClassifier train time = {} seconds".format(time.time() - start_time))

RandomForestClassifier train time = 168.16315174102783 seconds


In [91]:
start_time = time.time()
print(model.score(x_test, y_test))
print("RandomForestClassifier score time = {} seconds".format(time.time() - start_time))

0.9591953350361094
RandomForestClassifier score time = 5.7700629234313965 seconds


In [ ]:
from sklearn.svm import SVC

svclassifier = SVC(kernel='linear')
start_time = time.time()
svclassifier.fit(x_train, y_train)
print("SVC linear train time = {} seconds".format(time.time() - start_time))

In [ ]:
start_time = time.time()
print("SVC accuracy: ", accuracy_score(y_test, [round(item) for item in svclassifier.predict(x_test)]))
print("SVC accuracy time = {} seconds".format(time.time() - start_time))

In [ ]:
start_time = time.time()
print("SVC score: ", svclassifier.score(x_test, y_test))
print("SVC score time = {} seconds".format(time.time() - start_time))